In [1]:
import os
os.environ['DRJIT_LIBLLVM_PATH'] = 'C:\\Program Files\\LLVM\\bin\\LLVM-C.dll'
#print(os.environ['DRJIT_LIBLLVM_PATH'])
import sionna
import mitsuba as mi
import drjit as dr
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
from IPython.display import display, HTML
# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera, Antenna
# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement
from sionna.rt.solver_paths import SolverPaths as Solver
from Movement import *
from Visualization import *
from helper import *
from Channel import *

In [2]:
#tf.random.set_seed(10)
itu_metal_scattering_coefficient = 0.707 # To define the  scattering coefficient based on Sionna R = sqrt(1-S**2)
itu_concrete_scattering_coefficient = 0.707 # To define the  scattering coefficient based on Sionna R = sqrt(1-S**2)

c = 3e8 # m/s
robot_speed = 1.6 # Average speed of Robot
time_step = 0.3 # Time step to get the CIR

# tx and rx in cartesian coordinate
tx_position = [0,0.25,.94]
rx_position = [0,-0.25,.94]

# Below shows the initial and final positin of the Robot in cylindrical coordinate. r_min and r_max should be given
# theta will be calculated by the function accordingly to make sure that robot always is placed in the scene.

random_positions = False
robot_initial_position_radius = [3.5,4.5]  #  [r_min, [-theta , theta], z], [r_max, [-theta, theta], z] --> [r_min,r_max] 
robot_final_position_radius = [3,3.2]      #  [r_min, [-theta , theta], z], [r_max, [-theta, theta], z] --> [r_min,r_max] 

In [3]:
initial_position_cartesian , final_position_cartesian = robot_positions_solver(random_positions = random_positions,
                                                         robot_initial_position_radius = robot_initial_position_radius,
                                                         robot_final_position_radius = robot_final_position_radius)

Initial Position in cylindrical coordinate: [ 3.68221539 -0.01452084  0.        ]
Final Position in cylindrical coordinate: [ 3.0239578  -0.52800788  0.        ]
Initial Position in cartesian coordinate: [ 3.68182719 -0.05346699  0.        ]
Final Position in cartesian coordinate: [ 2.61213236 -1.52351085  0.        ]



In [4]:
robot_rotation(initial_position_cartesian, final_position_cartesian)

In [5]:
# Load integrated scene
#scene = load_scene('C:\\Users\\dehghani\\Documents\\test_scene\\Room_test.xml')
scene = load_scene('C:\\Users\\dehghani\\Documents\\test_scene\\scene.xml')

In [6]:
scene.frequency = 28e9
wavelength = c/scene.frequency
#print(scene.synthetic_array)
scene.synthetic_array = True # If set to False, ray tracing will be done per antenna element (slower for large arrays)

scene.tx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          pattern="iso",
                          polarization="V")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          pattern="iso",
                          polarization="V")

#scene.tx_array.show()

# Create transmitter
tx = Transmitter(name = "tx",
                 position = tx_position)

#tx.look_at([1.5,1,0.94]) 

# Add transmitter instance to scene
scene.add(tx)

# Create a receiver
rx = Receiver(name = "rx",
              position = rx_position)

#rx.look_at([1.5,1,0.94]) 

# Add receiver instance to scene
scene.add(rx)

# camera
my_cam = Camera("my_cam", position=[2.5,0,12], look_at=[2.5,0,0])
scene.add(my_cam)

In [7]:
for rm in scene.radio_materials.values():
    if rm.name=='itu_metal' :
        print(f'Before: rm.name: {rm.name} and   rm.scattering_coefficient = { rm.scattering_coefficient}')
        rm.scattering_coefficient= itu_metal_scattering_coefficient
        print(f'After: rm.name: {rm.name} and   rm.scattering_coefficient = { rm.scattering_coefficient}')
        
    if rm.name=='itu_concrete' :
        rm.scattering_coefficient = itu_concrete_scattering_coefficient

   #     rm.scattering_pattern = BackscatteringPattern(alpha_r=20, alpha_i=30, lambda_=0.5)

Before: rm.name: itu_metal and   rm.scattering_coefficient = 0.0
After: rm.name: itu_metal and   rm.scattering_coefficient = 0.7070000171661377


In [8]:
remove_scene_images() # Removing perivous images from the current directory for making a new gif for the new movement 

channel_realization_list = simulate_robot_movement(robot_speed = robot_speed, time_step = time_step,
                                                initial_position_cartesian = initial_position_cartesian,
                                                final_position_cartesian = final_position_cartesian, scene=scene)

create_gif_from_movement() #  Making a gif from the pictures made by simulate_robot_movement
gif_number = visualize_channel_list(channel_realization_list)

Time Step: 0.3 s
Robot Speed: 1.6 m/s 

Total Distance to be Traveled: 1.82 meters
Total Time to Complete the Movement: 1.14 seconds


In [9]:
channel_filename = f"channel_{gif_number}.gif" # Import the gif from directory 
movement_filename = f"movement_{gif_number}.gif" # Import the gif from directory 


html_str = f"""
<div style="display: flex; justify-content: center;">
    <div style="width: 75%;
    margin-right: 5px;">
        <img src="{channel_filename}" style="width: 100%;">
    </div>
    <div style="width: 35%;">
        <img src="{movement_filename}" style="width: 90%;">
    </div>
</div>
"""

# Display HTML with the synchronized GIFs
display(HTML(html_str))


In [10]:
# Concatenate all the series into on data frame with a common index
channel_realization_df = channel_df(channel_realization_list = channel_realization_list,
                                resolution_scale = 1, interpolation_method = 'linear')
channel_realization_df

,0.0,0.3,0.6,0.9
6.000000e-09,9.907422e-06+3.700572e-05j,-1.259278e-07+2.006182e-05j,-2.142079e-05-2.042549e-05j,-1.009526e-04+7.359205e-05j
7.000000e-09,1.484304e-05-7.273872e-06j,-7.445075e-05-4.077585e-05j,-6.311833e-05-3.295408e-05j,-3.602966e-05+6.637318e-05j
8.000000e-09,-7.840345e-06+1.387487e-05j,5.907240e-05+2.486292e-05j,2.268209e-05-2.666019e-05j,3.280693e-05+2.328306e-05j
9.000000e-09,1.195089e-05+1.895471e-05j,-1.415150e-05+3.975459e-06j,-3.487762e-05-1.721411e-05j,1.412103e-05-1.304787e-05j
1.000000e-08,-3.132370e-05+2.363717e-06j,-2.528389e-06-4.260794e-06j,2.593679e-05-1.292614e-06j,1.005937e-05+6.752749e-06j
1.100000e-08,4.704485e-06+1.018813e-07j,-1.000354e-05-2.316005e-06j,-1.345229e-05-9.234269e-06j,2.475527e-07-6.275577e-06j
1.200000e-08,8.451416e-06-1.732464e-06j,-7.636408e-06-2.242801e-06j,6.126157e-06-3.477088e-06j,-2.377656e-06+5.714646e-07j
1.300000e-08,4.390738e-06-9.289914e-07j,-2.736223e-07-2.636587e-07j,1.068986e-06-3.330608e-06j,5.266766e-06-7.845216e-08j
1.400000e-08,1.119048e-06+9.060060e-07j,-4.073240e-07-3.962959e-07j,7.227993e-07+1.114707e-06j,1.377908e-06-4.747474e-08j
1.500000e-08,1.850493e-07-8.081813e-07j,1.272683e-06-1.058103e-06j,1.382220e-06+9.922135e-08j,-1.522092e-07+1.659347e-07j


In [11]:
# Adding noise as  channel estimation error 
noise_power_percent = 0.1  # Power of the noise = variance 
noisy_channel_df = add_noise_to_channel(channel_df = channel_realization_df , noise_power_percent = noise_power_percent)
noisy_channel_df - channel_realization_df

,0.0,0.3,0.6,0.9
6.000000e-09,3.243797e-06+1.861006e-06j,4.335824e-06+5.750963e-06j,0.000001-0.000003j,-6.771315e-06-3.849908e-07j
7.000000e-09,3.398243e-06-6.758344e-07j,-4.453302e-06+7.607369e-06j,0.000002-0.000004j,4.325781e-06+2.523762e-06j
8.000000e-09,-3.545642e-06+1.601651e-06j,3.216170e-07+2.582767e-06j,-0.000001+0.000001j,2.013960e-06+2.265439e-06j
9.000000e-09,2.459675e-06-1.739349e-06j,-6.667704e-07-4.815587e-06j,0.000005+0.000003j,1.740557e-06+4.114287e-06j
1.000000e-08,-7.207952e-06-1.457306e-06j,8.203775e-06-2.226853e-06j,0.000000+0.000002j,-2.505615e-06+7.083333e-06j
1.100000e-08,-3.399911e-06-9.563166e-06j,7.191346e-06+5.625091e-06j,0.000002-0.000001j,1.100374e-06-2.219813e-06j
1.200000e-08,2.799963e-06-6.841157e-06j,7.650140e-06-2.146389e-06j,0.000002-0.000001j,1.437290e-07-6.850863e-06j
1.300000e-08,8.346233e-07-2.122127e-07j,-2.718165e-06+4.735768e-07j,-0.000002-0.000001j,7.980134e-07-1.715366e-06j
1.400000e-08,3.385818e-06+3.688440e-06j,2.873684e-06-5.369841e-06j,-0.000003-0.000005j,5.477605e-06+6.239711e-07j
1.500000e-08,-2.298914e-07+4.322684e-06j,-3.032695e-07+6.197522e-06j,-0.000004+0.000005j,2.764205e-06-2.388955e-06j


In [12]:
channel_magnitude_df = abs_complex_columns(noisy_channel_df)
channel_magnitude_df

,0.0,0.3,0.6,0.9
6.000000e-09,0.000041,2.615384e-05,3.100041e-05,1.302448e-04
7.000000e-09,0.000020,8.559204e-05,7.108878e-05,7.584144e-05
8.000000e-09,0.000019,6.542870e-05,3.339283e-05,4.318820e-05
9.000000e-09,0.000022,1.484207e-05,3.322914e-05,1.820436e-05
1.000000e-08,0.000039,8.619720e-06,2.617582e-05,1.576377e-05
1.100000e-08,0.000010,4.342638e-06,1.530751e-05,8.601661e-06
1.200000e-08,0.000014,4.389212e-06,9.335670e-06,6.664928e-06
1.300000e-08,0.000005,2.999143e-06,4.044841e-06,6.324503e-06
1.400000e-08,0.000006,6.271464e-06,4.057062e-06,6.879710e-06
1.500000e-08,0.000004,5.230046e-06,5.677427e-06,3.429919e-06


In [13]:
visualize_channel_df(channel_magnitude_df) # Make a gif out of noisy channel

In [14]:
channel_filename = f"channel_{gif_number}.gif" # Import the gif from directory 
noisy_channel_filename = f"noisy_channel_{gif_number}.gif" # Import the gif from directory 


html_str = f"""
<div style="display: flex; justify-content: center;">
    <div style="width: 50%;
    margin-right: 5px;">
        <img src="{channel_filename}" style="width: 100%;">
    </div>
    <div style="width: 50%;">
        <img src="{noisy_channel_filename}" style="width: 100%;">
    </div>
</div>
"""

# Display HTML with the synchronized GIFs
display(HTML(html_str))
